In [1]:
from codeocean.data_asset import DataAssetParams, DataAssetSearchParams,DataAssetAttachParams, Source, AWSS3Source
import pandas as pd
import os, sys
from codeocean import CodeOcean
from codeocean.data_asset import DataAssetParams, DataAssetSearchParams,DataAssetAttachParams, Source, AWSS3Source
client = CodeOcean(domain="https://codeocean.allenneuraldynamics.org", token=os.getenv("API_SECRET"))

In [3]:
# load and parse data ids
datalist_dir = '.../session_assets.csv'
data_df = pd.read_excel(datalist_dir, sheet_name='Sheet1')

In [13]:
col_to_attach = ['raw_data', 'sorted_curated']
curr_ids = list(data_df[col_to_attach[0]].values)
curr_ids

['2d28ccd9-48ab-429e-92ef-7723135ee306',
 'f7ceca5c-5010-4436-8a91-b0c780b04fd8']

In [22]:
# Lists of strings for id and mount
all_ids = []
all_mounts = []

for curr_col in col_to_attach:
    session_ids = list(data_df['session_id'].values)
    curr_ids = list(data_df[curr_col].values)
    curr_mount = [session_curr + '_' + curr_col for session_curr in session_ids]
    all_ids.extend(curr_ids)
    all_mounts.extend(curr_mount)


# Generate the list of DataAssetAttachParams objects
data_assets = [DataAssetAttachParams(id, mount) for id, mount in zip(all_ids, all_mounts)]
data_assets_id = all_ids

In [32]:
# Attach the generated list
results = client.capsules.attach_data_assets(
    capsule_id=os.getenv("CO_CAPSULE_ID"),
    attach_params=data_assets,
)

In [29]:
# Remove the generated list
results_rm = client.capsules.detach_data_assets(
    capsule_id=os.getenv("CO_CAPSULE_ID"),
    data_assets = list(data_assets_id)
)

In [8]:
directory = '/root/capsule/data'
recording_dirs = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
print(recording_dirs)

['/root/capsule/data/716325_2024-05-29_10-33-32', '/root/capsule/data/716325_2024-05-31_10-31-14', '/root/capsule/data/716325_2024-06-01_09-25-37', '/root/capsule/data/717121_2024-06-11_10-23-31', '/root/capsule/data/717121_2024-06-13_12-21-20', '/root/capsule/data/717121_2024-06-14_10-23-49', '/root/capsule/data/717121_2024-06-15_10-00-58', '/root/capsule/data/717121_2024-06-16_11-45-02', '/root/capsule/data/717259_2024-06-25_10-09-25', '/root/capsule/data/717259_2024-06-26_10-23-07', '/root/capsule/data/717259_2024-06-28_11-17-36']


In [10]:
AP_start = [None]*len(recording_dirs)
LFP_start = [None]*len(recording_dirs)
NI_start = [None]*len(recording_dirs)
AP_count = [None]*len(recording_dirs)
LFP_count = [None]*len(recording_dirs)
NI_count = [None]*len(recording_dirs)
session_id = [None]*len(recording_dirs)
local_sync_line = 1
for rec_ind, rec_dir in enumerate(recording_dirs):
    session_id[rec_ind] = os.path.basename(os.path.normpath(rec_dir))
    print(rec_dir)

    AP_key = 'ProbeA'

    session = Session(rec_dir)
    recording = session.recordnodes[0].recordings[0]
    events = recording.events
    for stream_ind in range(len(recording.continuous)):
        stream = recording.continuous[stream_ind]
        stream_name = stream.metadata['stream_name']
        print(stream_name)
        stream_source_node_id =stream.metadata['source_node_id']
        stream_freq = stream.metadata['sample_rate']
        stream_events = events[
                (events.stream_name == stream_name)
                & (events.processor_id == stream_source_node_id)
                & (events.line == local_sync_line)
                & (events.state == 1)
            ].sort_values(by='sample_number')
        sample_numbers = stream.sample_numbers

        if len(stream_events) == 0:
            stream_start_time = None
        else:
            stream_start_time = (stream_events.sample_number.values[0] - np.min(sample_numbers))/stream_freq

        if AP_key in stream.metadata['stream_name']:
            AP_start[rec_ind] = stream_start_time
            AP_count[rec_ind] = len(stream_events)
        if 'LFP' in stream.metadata['stream_name']:
            LFP_start[rec_ind] = stream_start_time
            LFP_count[rec_ind] = len(stream_events)
        if 'PXIe' in stream.metadata['stream_name']:
            NI_start[rec_ind] = stream_start_time
            NI_count[rec_ind] = len(stream_events)


import pandas as pd
sum_new = pd.DataFrame({
    'AP_start': AP_start,
    'LFP_start': LFP_start,
    'NI_start': NI_start,
    'AP_count': AP_count,
    'LFP_count': LFP_count,
    'NI_count': NI_count,
    'session_id': session_id
})

/root/capsule/data/716325_2024-05-29_10-33-32


NameError: name 'Session' is not defined

In [5]:
# but it can mount a data asset
import os
mouse_id = '608551'
session_date = '2022-05-28'
os.system('sh /code/mount_raw_data_asset.sh {} {}'.format(mouse_id,session_date))
print('done')

mouseid: 608551
sessiondate: 2022-05-28
1 data asset(s) found
attaching single-plane-ophys_608551_2022-05-28_14-56-19
done


now check /data if the needed data asset was attached